<a href="https://colab.research.google.com/github/ToiznerD/CloudComputing/blob/main/Project/QuestionsForm_Raz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from firebase import firebase
from google.colab import drive
import json
import ipywidgets as widgets
import time
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, db

def button_eventhandler(obj):
    global count, questions
    # Get number of question in  and increase the amount by 1
    if questionsFromDB.value == 'Pick a question to edit':
        count += 1
        questionsNum = count

    # If it is an update to an existing question in DB
    else:
        for i,q in enumerate(questions):
          if questionsFromDB.value == q[5]:
            questionsNum = i + 1

    # Saving correct answer in DB
    correct = ''
    match(int(correct_answer.value)):
        case 1:
          correct = answer_1.value
        case 2:
          correct = answer_2.value
        case 3:
          correct = answer_3.value
        case 4:
          correct = answer_4.value

    # Insert / update a question in DB
    accessDB.update({
        questionsNum : {
          'question' : question.value,
          'correct' : correct,
          'ans1' : answer_1.value,
          'ans2' : answer_2.value,
          'ans3' : answer_3.value,
          'ans4' : answer_4.value 
        }, 'Count': {'i': count}
    })

    # Update the local array of questions
    tempQ = [answer_1.value, answer_2.value, answer_3.value,
             answer_4.value, correct, question.value]
    
    questionsTemp = list(questionsFromDB.options)
    # Update the options in questions list and print correct message according to the action(insert/update)
    if questionsNum < count:
      questions[questionsNum] = tempQ
      questionsTemp[questionsNum] = tempQ[5]
      questionsFromDB.options = questionsTemp
      label.value = 'Question "' + tempQ[5] +'" was updated successfully in DB'
    else:
      questions.append(tempQ)
      questionsTemp.append(tempQ[5])
      questionsFromDB.options = questionsTemp
      label.value = 'Question "' + tempQ[5] + '" sent to DB successfully'

    reset_fields()
    time.sleep(5)
    label.value = ''

# Reset form's fields
def reset_fields():
    question.value = ''
    answer_1.value = ''
    answer_2.value = ''
    answer_3.value = ''
    answer_4.value = ''
    correct_answer.value = '1'
    questionsFromDB.value = 'Pick a question to edit'


# Fill the fields when picking a question to edit
def fill_fields(change):
  global questions
  # Check if default option is selected
  if change['new'] == 'Pick a question to edit':
      reset_fields()
      return

  # Find the question index and update the fields in the form
  for q in questions:
    if q[5] == change['new']:
      answer_1.value = q[0]
      answer_2.value = q[1]
      answer_3.value = q[2]
      answer_4.value = q[3]
      question.value = q[5]
      match(str(q[4])):
        case answer_1.value:
          correct = 1
        case answer_2.value:
          correct = 2
        case answer_3.value:
          correct = 3
        case answer_4.value:
          correct = 4
      correct_answer.value = str(correct)
      break

# Connect to DB
accessDB = db.reference('Questions/', app=firebase_admin.get_app(name="abc"))

#@title QuestionsForm
question = widgets.Textarea(description="Question:", layout=widgets.Layout(width='500px',height='100px'))
answer_1 = widgets.Text(description="Answer #1:", layout=widgets.Layout(width='300px'))
answer_2 = widgets.Text(description="Answer #2:", layout=widgets.Layout(width='300px'))
answer_3 = widgets.Text(description="Answer #3:", layout=widgets.Layout(width='300px'))
answer_4 = widgets.Text(description="Answer #4:", layout=widgets.Layout(width='300px'))

correct_answer = widgets.Dropdown(
    options=['1', '2', '3', '4'],
    value='1',
    description='Correct Answer:',
    disabled=False,
    layout=widgets.Layout(width='150px'),
    style={'description_width': 'initial'})

# Get questions from DB
questionsDict = accessDB.get()
questions = []
q = []
count = 0
for qKey in questionsDict:
  if qKey != 'Count':
    q.append(questionsDict[qKey]['ans1'])
    q.append(questionsDict[qKey]['ans2'])
    q.append(questionsDict[qKey]['ans3'])
    q.append(questionsDict[qKey]['ans4'])
    q.append(questionsDict[qKey]['correct'])
    q.append(questionsDict[qKey]['question'])
    questions.append(q)
    q = []
  else:
    count = int(questionsDict[qKey]['i'])

q.append('Pick a question to edit')
for qu in questions:
  q.append(qu[5])

questionsFromDB = widgets.Dropdown(
    options = q
)

add_edit_btn = widgets.Button(description='Add / Edit Question')
add_edit_btn.on_click(button_eventhandler)

label = widgets.Label(value='')

questionsFromDB.observe(fill_fields)

display(question, answer_1, answer_2, answer_3, answer_4, correct_answer, questionsFromDB, add_edit_btn, label)

Textarea(value='', description='Question:', layout=Layout(height='100px', width='500px'))

Text(value='', description='Answer #1:', layout=Layout(width='300px'))

Text(value='', description='Answer #2:', layout=Layout(width='300px'))

Text(value='', description='Answer #3:', layout=Layout(width='300px'))

Text(value='', description='Answer #4:', layout=Layout(width='300px'))

Dropdown(description='Correct Answer:', layout=Layout(width='150px'), options=('1', '2', '3', '4'), style=Desc…

Dropdown(options=('Pick a question to edit', 'How are me', 'How are me %$#^#', 'How are me $$@@@', 'fdfs', 'Wh…

Button(description='Add / Edit Question', style=ButtonStyle())

Label(value='')

In [11]:
#import firebase_admin
from firebase_admin import credentials, db


# Initialize the Firebase Admin SDK
cred = credentials.Certificate('/content/drive/My Drive/MyStyles/adminsdk.json')
firebase_admin.initialize_app(cred,{'databaseURL': 'https://cloudcomputing-e1f5d-default-rtdb.firebaseio.com/'}, name="abc")


print(db.reference('Users/', app=firebase_admin.get_app(name="abc")).get())

{'amit': {'password': '123', 'points': 440, 'username': 'amit'}, 'benshi': {'password': '123', 'points': 390, 'username': 'benshi'}, 'dor': {'password': '123', 'points': 600, 'username': 'dor'}, 'raz': {'password': '123', 'points': 460, 'username': 'raz'}}


In [8]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
